# AWS Machine Learning 목적별 가속기 튜토리얼
## [AWS Trainium](https://aws.amazon.com/machine-learning/trainium/)과 [AWS Inferentia](https://aws.amazon.com/machine-learning/inferentia/)를 [Amazon SageMaker](https://aws.amazon.com/sagemaker/)와 함께 사용하여 ML 워크로드를 최적화하는 방법 학습
## 파트 2/3 - SageMaker + [Hugging Face Optimum Neuron](https://huggingface.co/docs/optimum-neuron/index)을 사용하여 Trainum 인스턴스에서 Bert 모델 미세 조정하기

**SageMaker studio 커널: PyTorch 1.13 Python 3.9 CPU - ml.t3.medium** 

이 튜토리얼에서는 [HF Optimum Neuron](https://huggingface.co/docs/optimum-neuron/index)을 사용하여 [trn1 인스턴스](https://aws.amazon.com/ec2/instance-types/trn1/)에서 SageMaker로 미세 조정 작업을 시작하는 방법을 배우게 됩니다. HF Optimum Neuron은 학습 스크립트를 단순화하고 ML 개발자가 다양한 시나리오에서 재사용할 수 있는 이식 가능한 코드를 만들 수 있도록 도와주는 프레임워크입니다. 예를 들어 다양한 모델, 다양한 작업, 분산 학습(데이터 병렬, 텐서 병렬 등)에 활용할 수 있습니다. 또한 Optimum Neuron은 모델을 컴파일하고 AWS Inferentia에 배포하는 데 도움을 줍니다(이 튜토리얼의 3부에서 자세히 알아보세요).

섹션 02에서는 Optimum Neuron API에서 메타데이터를 추출하고 현재 테스트/지원되는 모델이 포함된 테이블을 렌더링하는 방법을 볼 수 있습니다(목록에 없는 유사한 모델도 호환될 수 있지만 직접 확인해야 합니다). 이 테이블은 어떤 모델을 선택하고 간단한 방식으로 미세 조정할 수 있는지 이해하는 데 중요합니다. 그러나 학습을 위해 모델을 선택하기 전에 **파트 3** 노트북에서 유사한 테이블을 확인하여 HF Optimum Neuron을 사용하여 AWS Inferentia에 배포할 수 있는 모델을 확인하세요. 이렇게 하면 엔드투엔드 솔루션을 계획하고 지금 바로 구현을 시작할 수 있습니다.

## 1) 필요한 패키지 설치

In [ ]:
%pip install -r requirements.txt

## 2) 지원되는 모델/작업

이름 뒤에 **[TP]**가 있는 모델은 텐서 병렬 처리를 지원합니다

In [ ]:
from IPython.display import Markdown, display

display(Markdown("../docs/optimum_neuron_models.md"))

## 3) SageMaker와 HF Optimum Neuron을 사용하여 모델 미세 조정하기
입력 이메일이 스팸인지 아닌지 예측하는 텍스트 분류기로 Bert 모델을 학습시키고 있습니다. 자신의 시나리오에 맞게 조정하려면 다음 변수만 변경하세요: **MODEL**과 **TASK**는 위 표를 참조하세요.
  - MODEL: HF 포털에서 사용 가능한 모델의 이름. 위 표에서 원하는 "모델 이름"을 클릭하여 해당 특정 모델에 대한 모든 옵션을 나열합니다.
  - TASK: 위 표에서 원하는 작업(열 이름)을 복사합니다. 선택한 모델이 해당 작업을 지원하는지 확인하세요. 그렇지 않으면 모델을 변경해야 합니다.

**이 샘플을 실행하려면 Hugging Face 자격 증명과 사용자 지정 저장소가 필요합니다.** 이 구성은 모델의 캐시 파일을 저장하는 데 필요합니다. [huggingface.co](huggingface.co/)로 이동하여 필요한 경우 계정을 만드세요. 또한 **액세스 토큰**과 새 모델 저장소를 생성해야 합니다.

**CUSTOM_CACHE_REPO**를 이 학습 작업을 위해 생성한 모델 저장소로 설정하세요. 예: **user-name/model-name**. 아직 캐시 저장소가 없다면 [이 페이지의 지침을 따라](https://huggingface.co/docs/optimum-neuron/guides/cache_system) 하나 만드세요. **HF_TOKEN**을 계정에서 생성한 유효한 Hugging Face 액세스 토큰으로 설정하세요.

**HF_CACHE_REPO**와 **HF_TOKEN**을 설정하지 않으면 학습 작업을 호출할 때마다 모델이 다시 컴파일되어 시간이 소요됩니다. 이 단계를 최적화하기 위해 캐시 메커니즘을 사용하는 것이 **매우** 권장됩니다.

In [ ]:
# Click on the "model name" in the table above to visualize which options of models you have to fine-tune
# i.e: If you click on bert, bert-base-uncased is an available option to select
MODEL="bert-base-uncased"
TASK="SequenceClassification"
HF_CACHE_REPO="aws-neuron/optimum-neuron-cache"
HF_TOKEN=None
assert len(MODEL)>0, "Please, use the table above to define a valid model name"
assert len(TASK)>0, "Please, use the table above to define a valid model task"

In [ ]:
import os
import boto3
import sagemaker

print(sagemaker.__version__)
if not sagemaker.__version__ >= "2.146.0": print("You need to upgrade or restart the kernel if you already upgraded")

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = sess.boto_region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

if not os.path.isdir('src'): os.makedirs('src', exist_ok=True)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

### 3.1) SageMaker에서 호출할 학습 스크립트

이 학습 스크립트는 프로세스를 단순화하기 위해 HF Optimum Neuron API를 사용합니다. [여기에서 자세히 알아볼 수 있습니다](https://huggingface.co/docs/optimum-neuron/quickstart). 이 스크립트는 학습 작업을 준비하고 모델을 빠르게 미세 조정하는 방법을 보여주기 위한 것입니다. 필요에 따라 이 스크립트를 조정/수정해야 할 수 있습니다.

In [ ]:
!pygmentize src/train.py

In [ ]:
!pygmentize src/requirements.txt

### 3.2) SageMaker Estimator 정의하기
이 객체는 학습 작업을 구성하고 필요한 하이퍼파라미터 및 기타 구성 설정을 설정하는 데 도움이 됩니다.

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="train.py", # Specify your train script
    source_dir="src",
    role=role,
    sagemaker_session=sess,
    instance_count=1,
    instance_type='ml.trn1.2xlarge',
    disable_profiler=True,
    output_path=f"s3://{bucket}/output",
    image_uri=f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training-neuronx:1.13.1-neuronx-py310-sdk2.18.0-ubuntu20.04",
    
    # Parameters required to enable checkpointing
    # This is necessary for caching XLA HLO files and reduce training time next time    
    checkpoint_s3_uri=f"s3://{bucket}/checkpoints/{MODEL}",
    volume_size = 512,
    distribution={
        "torch_distributed": {
            "enabled": True
        }
    },
    environment={
        # Uncomment the following line to precompile the cache files
        # "RUN_NEURON_PARALLEL_COMPILE": "1"
        "OMP_NUM_THREADS": "1",
        "FI_EFA_FORK_SAFE": "1",        
        "NEURON_RT_STOCHASTIC_ROUNDING_EN": "1",        
        "MALLOC_ARENA_MAX":"80", # required to avoid OOM

        # Uncomment the following line if you defined a HF HUB cache repo
        "CUSTOM_CACHE_REPO": HF_CACHE_REPO
    },
    hyperparameters={
        "model_id": MODEL,
        "task": TASK,        
        "bf16": True,
        "zero_1": True,
        
        "learning_rate": 5e-5,
        "epochs": 1,
        "train_batch_size": 4,
        "eval_batch_size": 4,
        "max_sen_len": 256, # this needs to be aligned with the sentence len used in the data preparation

        # Uncomment this line if you have defined a valid HF_TOKEN
        #"hf_token": HF_TOKEN,
        
        # Uncomment and configure the following line to enable TP
        #"tensor_parallel_size": 8,        
    },
    metric_definitions=[        
        {"Name": "eval_loss", "Regex": ".eval_loss.:\S*(.*?),"},
        {"Name": "train_loss", "Regex": "'loss.:\S*(.*?),"},
        {"Name": "it_per_sec", "Regex": ",\S*(.*?)it.s."},
    ]
)
#if not HF_TOKEN is None and len(HF_TOKEN) > 0:
    
estimator.framework_version = '1.13.1' # workround when using image_uri

In [ ]:
train_uri=f"s3://{bucket}/datasets/spam/train"
eval_uri=f"s3://{bucket}/datasets/spam/eval"
print(f"{train_uri}\n{eval_uri}")

In [ ]:
estimator.fit({"train": train_uri, "eval": eval_uri})

In [ ]:
with open("training_job_name.txt", "w") as f:
    f.write(estimator._current_job_name)

## 4) 이제 모델을 배포할 시간입니다

[Inf2에서의 배포/추론 노트북 열기](03_ModelInference.ipynb)  
[Inf1에서의 배포/추론 노트북 열기](03_ModelInferenceInf1.ipynb)